In [1]:
%reload_ext autoreload
%autoreload 2
import os
import re
import sys
import glob
import numpy as np
import tensorflow as tf

In [3]:
libdir = '/home/tkurihana/scratch-midway2/clouds/src_analysis/lib_hdfs'
sys.path.insert(1,os.path.join(sys.path[0],libdir))
from lib_modis02 import load_tfrecord
from lib_modis02 import _get_imgs
from lib_modis02 import _get_num_imgs

In [26]:
from tensorflow.python.keras.layers import *
from tensorflow.python.keras.models import Model, Sequential

In [35]:
libdir = '/home/tkurihana/Clouds_dnn/driver'
sys.path.insert(1,os.path.join(sys.path[0],libdir))
from models import resblock
from models import scale_change_block

### Load model
How to load `ckpt` format data?   
Helpful resource[TnedorFlow official-document](https://www.tensorflow.org/beta/tutorials/keras/save_and_restore_models?hl=ja) 

*Problem*  
ONLY latest 5 checkpoints are  saved (others are supposed to be deleted)

In [4]:
model_dir = '/project2/foster/clouds/output/mod02/m02_dgx_0725_n128_29_31'

In [5]:
### latest model weight
latest = tf.train.latest_checkpoint(model_dir)
latest

'/project2/foster/clouds/output/mod02/m02_dgx_0725_n128_29_31/model.ckpt-3'

load model structure

- Helpful web resoueces  
[Hatena blog; TF & Jupyter notebook issue for saver](http://mizti.hatenablog.com/entry/2017/05/20/220645)  
[MetaFlow; TensorFlow: saving/restoring and mixing multiple models](https://blog.metaflow.fr/tensorflow-saving-restoring-and-mixing-multiple-models-c4c94d5d7125) --> Check `Restoring the weights`  

In [42]:
metadata = model_dir+'/model.ckpt-3.meta'
tf.reset_default_graph() # <----- This is key line! DO NOT erase here to prevent notebook from automatically asssgin new vars 
with tf.Session() as sess:  
    saver = tf.train.import_meta_graph(metadata)
    saver.restore(sess,tf.train.latest_checkpoint(model_dir))

INFO:tensorflow:Restoring parameters from /project2/foster/clouds/output/mod02/m02_dgx_0725_n128_29_31/model.ckpt-3


In [43]:
saver

Conver tf-based checkpoint saver model to keras based model  
- Helpful web resources  
[Stackoverflow; how can I convert a trained tf model to keras.](https://stackoverflow.com/questions/44466066/how-can-i-convert-a-trained-tensorflow-model-to-keras)

In [44]:
metadata = model_dir+'/model.ckpt-3.meta'
tf.reset_default_graph() # <----- This is key line! DO NOT erase here to prevent notebook from automatically asssgin new vars 
with tf.Session() as sess:  
    saver = tf.train.import_meta_graph(metadata)
    saver.restore(sess,tf.train.latest_checkpoint(model_dir))
    # get all global variables (including model variables)
    vars_global = tf.global_variables()
    # get their name and value and put them into dictionary
    sess.as_default()
    model_vars = {}
    for var in vars_global:
        try:
            model_vars[var.name] = var.eval()
        except:
            print("For var={}, an exception occurred".format(var.name))

INFO:tensorflow:Restoring parameters from /project2/foster/clouds/output/mod02/m02_dgx_0725_n128_29_31/model.ckpt-3


In [45]:
vars_global

[<tf.Variable 'global_step:0' shape=() dtype=int64_ref>,
 <tf.Variable 'autoencoder/conv2d/kernel:0' shape=(3, 3, 6, 16) dtype=float32>,
 <tf.Variable 'autoencoder/conv2d/bias:0' shape=(16,) dtype=float32>,
 <tf.Variable 'autoencoder/encoding_0/conv2d_1/kernel:0' shape=(3, 3, 16, 16) dtype=float32>,
 <tf.Variable 'autoencoder/encoding_0/conv2d_1/bias:0' shape=(16,) dtype=float32>,
 <tf.Variable 'autoencoder/encoding_0/conv2d_2/kernel:0' shape=(3, 3, 16, 16) dtype=float32>,
 <tf.Variable 'autoencoder/encoding_0/conv2d_2/bias:0' shape=(16,) dtype=float32>,
 <tf.Variable 'autoencoder/encoding_0/batch_normalization/gamma:0' shape=(16,) dtype=float32>,
 <tf.Variable 'autoencoder/encoding_0/batch_normalization/beta:0' shape=(16,) dtype=float32>,
 <tf.Variable 'autoencoder/encoding_0/batch_normalization/moving_mean:0' shape=(16,) dtype=float32>,
 <tf.Variable 'autoencoder/encoding_0/batch_normalization/moving_variance:0' shape=(16,) dtype=float32>,
 <tf.Variable 'autoencoder/encoding_1/conv2d

In [47]:
with tf.Session() as sess:
    sess.as_default()
    for var in vars_global:
        a = var.eval()
        print(a);stop

FailedPreconditionError: Attempting to use uninitialized value global_step
	 [[{{node _retval_global_step_0_0}} = _Retval[T=DT_INT64, index=0, _device="/job:localhost/replica:0/task:0/device:CPU:0"](global_step)]]